# Competencia Kaggle


Vamos a participar en la competencia de Kaggle Titanic - Machine Learning from Disaster.


Realizaremos los pasos ya vistos para analizar y procesar los datos, luego haremos un modelo base con XGboost e intentaremos mejorarlo modificando algunos hiperparámetros. 


Finalmente enviaremos el listado de pasajeros con su predicción de clase y veremos qué tan bien nos ha ido.


Para ingresar a la competencia, deben registrarse en Kaggle e ir a: https://www.kaggle.com/competitions/titanic/overview, en esa página deben hacer click en Join Competition.

# Datos

Los datos los encontrarán en la pestaña Datos de la competencia. Ahí podran bajar los datasets y además ver una descripción de las variables. 

La competencia provee el dataset de entrenamiento con su target y el dataset de validacion/test sin el target. Habrá que hacer la predicción sobre el dataset de validación/test y enviar esas predicciones.

# EDA exploratory data analysis

Deberán hacer el análisis general del dataset y realizar el preprocesamiento adecuado (limpieza).

Luego deberán analizar las variables una por una y decidir cuáles son candidatas a permanecer en el modelo y cuales deberían ser eliminadas.


# Feature selection 

Cuando tengan el dataset limpio y esté hecho el análisis de las variables. Es hora de elegir aquellas que permanecerán en el modelo y realizar las transformaciones necesarias para que puedan procesarse con el algoritmo.

# Model selection

Para acelerar el proceso, esta vez utilizaremos desde el inicio el algoritmo XGBoost.

Haremos una versión base del modelo sin modificar parámetros.

# Model tunning

En base a los resultados del modelo base, podemos comenzar a modificar parámetros o incluso ver si es necesario modificar las variables iniciales.

# Modelo final e implementación de la solución

Cuando el modelo final esté listo, plotearemos las métricas y las variables mas significativas. 

Además, produciremos el listado de pasajeros con su predicción y lo subiremos a Kaggle para obtener una puntuación.

# Vamos a ello

In [1]:
#  Importamos Pandas como siempre
import pandas as pd
# Importamos XGBOOST que es el algoritmo que vamos a utilizar
import xgboost as xgb
# Importamos las funciones de métricas para evaluar el modelo
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Lo primero es subir a Dataframes nuestros datasets descargados de Kaggle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [77]:
df_train = pd.read_csv('/content/drive/MyDrive/train.csv')
df_test= pd.read_csv('/content/drive/MyDrive/test.csv')

# Vemos que hay dentro de cada dataset y solucionamos los nulos de la manera con las ideas que vimos en clase.

Para este ejemplo simplemente imputaré la mediana y eliminaré las columnas con muchos nulos.


In [22]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [23]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
df_train.shape

(891, 12)

In [16]:
df_train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [33]:
# Vemos los nulos
df_train.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [25]:
# Tratamos los nulos que nos interesan (Age)
df_train['Age'] = df_train['Age'].fillna(value=df_train['Age'].mean())

In [44]:
# Quito las columnas que no voy a utilizar como PassengerId, ticket y Cabin a falta de un método mejor (luego cuando vayan a mejorar el primer score, prueben ideas vistas en clase)
df_train = df_train.drop(['PassengerId', 'Ticket', 'Cabin', 'Name'], axis=1)

KeyError: ignored

In [45]:
df_train = df_train.drop(['Name'], axis=1)

In [32]:
# Dropeamos las dos filas que tienen nulos en Embarked
df_train = df_train.dropna(subset=['Embarked'])

In [46]:
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


# Ahora con nuestros datos limpios, vamos a crear nuestra matriz de variables X_train y el vector de target y_train

In [49]:
y_train = df_train['Survived']
X_train = df_train.drop(columns='Survived')

In [50]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    889 non-null    int64  
 1   Sex       889 non-null    object 
 2   Age       889 non-null    float64
 3   SibSp     889 non-null    int64  
 4   Parch     889 non-null    int64  
 5   Fare      889 non-null    float64
 6   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 55.6+ KB


In [51]:
y_train.shape

(889,)

# Hacemos One Hot Encoding en nuestra X_train, sin elegir columnas ya que en este caso no es necesario. La función solo modifica variables categóricas y no a las numéricas.

In [52]:
X_train = pd.get_dummies(X_train, drop_first=True)

In [53]:
X_train.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,1,0,1
1,1,38.0,1,0,71.2833,0,0,0
2,3,26.0,0,0,7.9250,0,0,1
3,1,35.0,1,0,53.1000,0,0,1
4,3,35.0,0,0,8.0500,1,0,1


# Llamamos al algoritmo y lo instanciamos dentro de una variable con un nombre representativo. No ingresasamos ningún parámetro específico en esta instancia, usaremos los valores por defecto.

In [54]:
import xgboost as xgb
xgbc_base = xgb.XGBClassifier(random_state=0)

In [ ]:
# Llamamos a la variable clasificador y entrenamos con los datos de X_train e y_train


In [55]:
xgbc_base.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=0, ...)

# El modelo ha quedado entrenado y guardado dentro de la variable Clasificador

In [57]:
# Podemos hacer una predicción con los datos de X_train para ver que score nos arroja
y_pred = xgbc_base.predict(X_train)

# Usamos la matriz de confusión y la métrica AUC para saber que tan bien predice con los datos de Train

In [58]:
from sklearn.metrics import confusion_matrix

In [60]:
confusion_matrix(y_train, y_pred)

array([[540,   9],
       [ 23, 317]])

# Con estos resultados a priori podríamos decir que el clasificador hace un buen trabajo. Pero sin contar con las etiquetas de target del set de Test no podemos comparar los dos scores y saber si hay o no sobreentrenamiento.

# Ahora haremos la predicción con los datos del X_test y a esa predicción la uniremos a los PassengerIds para enviar a Kaggle y que ellos hagan la comparación con el y_test y nos revuelvan el resultado y nuestro puesto en el ranking.

In [61]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [62]:
df_test['Age'] = df_test['Age'].fillna(value=df_test['Age'].mean())

In [63]:
df_test['Fare'] = df_test['Fare'].fillna(value=df_test['Fare'].mean())

In [64]:
df_test = df_test.drop(['PassengerId', 'Ticket', 'Cabin', 'Name'], axis=1)

In [70]:
df_test.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,3,34.5,0,0,7.8292,1,1,0
1,3,47.0,1,0,7.0000,0,0,1
2,2,62.0,0,0,9.6875,1,1,0
3,3,27.0,0,0,8.6625,1,0,1
4,3,22.0,1,1,12.2875,0,0,1


In [69]:
df_test = pd.get_dummies(df_test, drop_first=True)

In [71]:
test_pred = xgbc_base.predict(df_test)

In [73]:
survived_series = pd.Series(test_pred, name='Survived')

In [ ]:
# Convierto las predicciones en una serie de pandas para poder usar concat() con la serie de PassengerId


In [78]:
submition = pd.concat([df_test['PassengerId'], survived_series], axis=1)

In [ ]:
# Concateno ambas series y las bajo a un csv para poder hacer el submition.


In [82]:
submition.to_csv('/content/drive/MyDrive/submition.csv', header=True, index=False)